In [7]:
import pymysql as sql
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
from sklearn import cluster, preprocessing
import math
import os
import rds_config
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline
from __future__ import division
plt.rcParams['figure.figsize'] = (20.0, 10.0)

import warnings
warnings.filterwarnings('ignore')

import helper_functions

In [8]:
# open database
engine = helper_functions.create_db_engine()

In [9]:
df_player_season_stats = pd.read_sql('''SELECT g.Player, g.Height, g.Weight, ps.*, p.PTSPerG, p.3PAPerG, a.FTr
                                        FROM BasketballDatabase.PlayerSeasonShootingStatsYTD ps
                                        JOIN BasketballDatabase.PlayerSeasonStatsPerGameYTD p on p.PlayerID = ps.PlayerID
                                        JOIN BasketballDatabase.PlayerSeasonAdvStatsYTD a on a.PlayerID = ps.PlayerID
                                        JOIN BasketballDatabase.GeneralPlayerInfo g on g.PlayerID = ps.PlayerID
                                        ''', con=engine)
df_player_season_stats = df_player_season_stats.drop_duplicates(subset=['PlayerID'], keep=False).reset_index(drop=True)

df_player_season_stats['Height'] = df_player_season_stats['Height'].apply(lambda x: int(x.split('-')[0])*12 + int(x.split('-')[1]))

df_player_season_stats.drop(['FGPercent', 'AvgShotDist', 'Season', 'GameTypeID', 'TeamID'], axis=1, inplace=True)
df_player_season_stats

,Player,Height,Weight,PlayerID,Age,2PAPercent,PercentFGA0to2ft,PercentFGA3to9ft,PercentFGA10to15ft,PercentFGA16Plusftto3,...,FGPercent10to15ft,FGPercent16Plusftto3,3PFGPercent,Percent2PAAstByOthers,Percent3PAAstByOthers,Percent3PAFromCorner,3PPercentFromCorner,PTSPerG,3PAPerG,FTr
0,Jason Terry,74,185,5,39,0.288,0.119,0.062,0.049,0.058,...,0.364,0.308,0.410,0.586,0.939,0.534,0.384,4.1,2.3,0.128
1,Metta World Peace,78,260,25,37,0.414,0.172,0.034,0.069,0.138,...,0.500,0.000,0.118,0.000,1.000,0.235,0.000,1.1,0.9,0.414
2,Jamal Crawford,77,200,32,36,0.630,0.092,0.124,0.120,0.298,...,0.410,0.434,0.359,0.215,0.595,0.172,0.358,12.4,4.0,0.231
3,Dirk Nowitzki,84,245,46,38,0.685,0.064,0.056,0.213,0.353,...,0.463,0.450,0.384,0.774,1.000,0.056,0.455,14.4,4.0,0.172
4,Mike Miller,80,218,156,36,0.133,0.000,0.067,0.000,0.067,...,0.000,0.000,0.462,0.000,1.000,0.077,0.000,1.2,0.8,0.133
5,Vince Carter,78,220,216,40,0.408,0.188,0.095,0.048,0.078,...,0.182,0.361,0.369,0.443,0.871,0.201,0.364,8.0,4.0,0.240
6,Paul Pierce,79,235,283,39,0.403,0.097,0.016,0.129,0.161,...,0.500,0.300,0.351,0.545,1.000,0.135,0.600,3.2,1.7,0.194
7,Joe Johnson,79,240,443,35,0.595,0.079,0.229,0.126,0.161,...,0.431,0.348,0.384,0.146,0.865,0.332,0.416,8.8,3.2,0.124
8,Zach Randolph,81,260,450,35,0.902,0.296,0.287,0.107,0.211,...,0.402,0.377,0.225,0.482,1.000,0.146,0.308,14.1,1.3,0.200
9,Richard Jefferson,79,233,455,36,0.460,0.298,0.086,0.025,0.052,...,0.500,0.176,0.335,0.736,0.983,0.472,0.386,5.8,2.4,0.304


In [10]:
data = df_player_season_stats[['PercentFGA0to2ft', 'PercentFGA3to9ft', 'PercentFGA10to15ft',
                               'PercentFGA16Plusftto3', 'PercentFGA3P', 'Percent2PAAstByOthers',
                               'Percent3PAAstByOthers', 'Percent3PAFromCorner', 'FTr', 'Height', 'Weight']]

data = helper_functions.standard_scaler(data)

rand_state = 25
y_pred = cluster.KMeans(n_clusters=6, max_iter=1000, n_init=20, random_state=rand_state).fit_predict(data)
df_class = pd.DataFrame({'Class': y_pred})

In [11]:
df_player_season_stats = pd.concat([df_player_season_stats, df_class], axis=1)
df_player_season_stats

,Player,Height,Weight,PlayerID,Age,2PAPercent,PercentFGA0to2ft,PercentFGA3to9ft,PercentFGA10to15ft,PercentFGA16Plusftto3,...,FGPercent16Plusftto3,3PFGPercent,Percent2PAAstByOthers,Percent3PAAstByOthers,Percent3PAFromCorner,3PPercentFromCorner,PTSPerG,3PAPerG,FTr,Class
0,Jason Terry,74,185,5,39,0.288,0.119,0.062,0.049,0.058,...,0.308,0.410,0.586,0.939,0.534,0.384,4.1,2.3,0.128,0
1,Metta World Peace,78,260,25,37,0.414,0.172,0.034,0.069,0.138,...,0.000,0.118,0.000,1.000,0.235,0.000,1.1,0.9,0.414,0
2,Jamal Crawford,77,200,32,36,0.630,0.092,0.124,0.120,0.298,...,0.434,0.359,0.215,0.595,0.172,0.358,12.4,4.0,0.231,1
3,Dirk Nowitzki,84,245,46,38,0.685,0.064,0.056,0.213,0.353,...,0.450,0.384,0.774,1.000,0.056,0.455,14.4,4.0,0.172,3
4,Mike Miller,80,218,156,36,0.133,0.000,0.067,0.000,0.067,...,0.000,0.462,0.000,1.000,0.077,0.000,1.2,0.8,0.133,0
5,Vince Carter,78,220,216,40,0.408,0.188,0.095,0.048,0.078,...,0.361,0.369,0.443,0.871,0.201,0.364,8.0,4.0,0.240,0
6,Paul Pierce,79,235,283,39,0.403,0.097,0.016,0.129,0.161,...,0.300,0.351,0.545,1.000,0.135,0.600,3.2,1.7,0.194,0
7,Joe Johnson,79,240,443,35,0.595,0.079,0.229,0.126,0.161,...,0.348,0.384,0.146,0.865,0.332,0.416,8.8,3.2,0.124,3
8,Zach Randolph,81,260,450,35,0.902,0.296,0.287,0.107,0.211,...,0.377,0.225,0.482,1.000,0.146,0.308,14.1,1.3,0.200,3
9,Richard Jefferson,79,233,455,36,0.460,0.298,0.086,0.025,0.052,...,0.176,0.335,0.736,0.983,0.472,0.386,5.8,2.4,0.304,0


In [18]:
df_player_season_stats[df_player_season_stats['Class'] == 5].sort(['PTSPerG'], ascending=False)

,Player,Height,Weight,PlayerID,Age,2PAPercent,PercentFGA0to2ft,PercentFGA3to9ft,PercentFGA10to15ft,PercentFGA16Plusftto3,...,FGPercent16Plusftto3,3PFGPercent,Percent2PAAstByOthers,Percent3PAAstByOthers,Percent3PAFromCorner,3PPercentFromCorner,PTSPerG,3PAPerG,FTr,Class
144,Hassan Whiteside,84,265,1099,27,1.000,0.440,0.381,0.102,0.077,...,0.362,0.000,0.586,0,0.0,0,16.8,0.0,0.358,5
201,Andre Drummond,83,279,1238,23,0.992,0.500,0.414,0.070,0.005,...,0.000,0.286,0.610,0,0.0,0,14.0,0.1,0.389,5
210,Jonas Valanciunas,84,265,1251,24,1.000,0.457,0.290,0.143,0.111,...,0.384,0.000,0.614,0,0.0,0,12.0,0.0,0.321,5
156,Greg Monroe,83,265,1117,26,0.997,0.572,0.299,0.056,0.069,...,0.426,0.000,0.584,0,0.0,0,11.8,0.0,0.334,5
304,Jahlil Okafor,83,275,1479,21,1.000,0.408,0.374,0.161,0.055,...,0.269,0.000,0.426,0,0.0,0,11.8,0.0,0.335,5
249,Steven Adams,84,255,1355,23,0.997,0.555,0.391,0.043,0.005,...,0.000,0.000,0.681,0,0.0,0,11.5,0.0,0.401,5
86,Marcin Gortat,83,240,956,32,0.997,0.528,0.310,0.137,0.024,...,0.400,0.000,0.725,0,0.0,0,10.7,0.0,0.237,5
101,Robin Lopez,84,255,985,28,0.997,0.302,0.337,0.097,0.261,...,0.452,0.000,0.682,0,0.5,0,10.2,0.0,0.129,5
27,Al Jefferson,82,289,664,32,0.998,0.289,0.380,0.217,0.108,...,0.353,0.000,0.536,0,0.0,0,8.1,0.0,0.180,5
330,Willie Cauley-Stein,84,240,1510,23,0.995,0.464,0.327,0.120,0.084,...,0.314,0.000,0.734,0,0.0,0,7.7,0.0,0.337,5
